Full duplex radio를 위한 연습



In [ ]:
import numpy as np
import tensorflow as tf

: 

$s[n]$ : transmitted signal at time $n$, (real number)  
시간에 따른 송신신호

$h_p[n]$ : the channel coefficient of the $p$-th order  
p차 채널 계수

$P$ : the largest order


\begin{align}
y[n]= \sum_\limits{p=1,p=odd}^{P}\sum\limits_{l=0}^{L} h_{p}[l]s[n-l]^{p}.      ~~~~~~~~~~ (1)
\end{align}

In [ ]:
length_data = 100000  # length of s[n] | 송신신호 크기
L = 3                 # length of delay profile [h[0], h[1], ..., h[L-1]] | 딜레이
var_S = 5e-1          # variance of transmitted signal => 0.5 | 송신신호 값
var_N = 1e-5          # variance of noise => 0.00001 | 노이즈 값

P_max = 5             # the largest order
P_min = 1             # the smallest order 

# traning set 
s = np.sqrt(var_S)*np.random.randn(length_data)     # the transmitted signal => matrix(100000,1) 평균:0 표편:np.sqrt(0.5)
y = np.zeros(length_data + L -1,dtype='float32')    # the received signal => matrix(100002,1)
# y = (0., 0., 0., 0., ... 0.) length = 100002

#########################################################
##  Training Data Set
########################################################

## Received symbols according to (1) | (1)에 따른 기호 수신
h = np.random.randn(P_max+1,L)  # matrix(6,3) 평균:0, 표편:1
for p in [a for a in range(P_min,P_max+1) if a%2==1]:
    y = y + np.convolve(h[p],np.power(s,p)) #np.power(s,p) = s^p
    #np.convolve([1, 2, 3], [0, 1, 0.5]) => (3,2,1)*(0,1,0.5) = (0., 1., 2.5., 4., 1.5)

### additive noise ### 노이즈 추가
y = y + np.sqrt(var_N)*np.random.randn(length_data + L-1) # 평균:y ,표편:np.sqrt(0.5) , 난수: 100002개

In [ ]:
# s의 시각화
import matplotlib.pyplot as plt

plt.style.use('default')
plt.rcParams['figure.figsize'] = (6, 3)
plt.rcParams['font.size'] = 12

plt.hist(s, bins=100, density=True, alpha=0.5, histtype='step', label='(mean, stddev)=(0, np.sqrt(0.5))')

plt.xlim(-5, 5)
plt.legend()
plt.show()

In [ ]:
# y의 시각화
import matplotlib.pyplot as plt

plt.style.use('default')
plt.rcParams['figure.figsize'] = (6, 3)
plt.rcParams['font.size'] = 12

plt.hist(y, bins=100, density=True, alpha=0.5, histtype='step', label='(mean, stddev)=(0, np.sqrt(0.5))')

plt.xlim(-25, 25)
plt.legend()
plt.show()

In [ ]:
print(h)

Our objective is to estimate $\{h_p[0], h_p[1], h_p[2]\}$ for $p = 1, 3, 5$ assuming we do not know them.   
We will apply the linear regression method first to estimate $\{h_p[0], h_p[1], h_p[2]\}$. 


$y[n]$ is affected by $\{s[n], s[n-1], s[n-2], s^3[n], s^3[n-1], s^3[n-2], s^5[n], s^5[n-1], s^5[n-2]\}$.  
We can generate training set such as

$s[2], s[1], s[0], s^3[2], s^3[1], s^3[0], s^5[2], s^5[1], s^5[0], y[2]$  
$s[3], s[2], s[1], s^3[3], s^3[2], s^3[1], s^5[3], s^5[2], s^5[1], y[3]$  
$s[4], s[3], s[2], s^3[4], s^3[3], s^3[2], s^5[4], s^5[3], s^5[2], y[4]$  
$\vdots$  $\vdots$  
where the last element of each row is a target data and the rest of the elements of each row is feature data. 




In [ ]:
## Training data generation 
x_data = np.zeros([length_data - L+ 1, L],dtype = 'float32')   ## feature data
# 1[0., 0., 0., ..., 0., 0.]
# 2[0., 0., 0., ..., 0., 0.]
# .[          ...          ]
# L[0., 0., 0., ..., 0., 0.]
y_data = np.zeros([length_data - L + 1,1],dtype = 'float32')   ## target data
# 1[0., 0., 0., ..., 0., 0.]

for i in range(length_data - L + 1):
    y_data[i] = y[i + L -1] # y_data[0] = y[2], y_data[1] = y[3], ...
    x_data[i] = s[i : i + L] # 유사 sliding window filter | x_data[0] = s[0:3], x_data[1] = s[1:4], ...

x_data = np.concatenate((x_data, np.power(x_data,3), np.power(x_data,5)),axis=1) 
# concatenate axis = 0면 아래, axis = 1면 오른쪽 붙이기
# x_data, (x_data)^3, (x_data)^5
train_data = x_data


In [ ]:
s.shape

We also generate test data set. 

In [ ]:
###################################################    
## Test Data Set
####################################################
len_test_data = 100000  # length of s[n] for test data set 

y = np.zeros(len_test_data + L -1,dtype='float32')
s = np.sqrt(var_S)*np.random.randn(len_test_data) 

h = np.random.randn(P_max+1,L)
for p in [a for a in range(P_min,P_max+1) if a%2==1]:
    y = y + np.convolve(h[p],np.power(s,p))
y = y + np.sqrt(var_N)*np.random.randn(length_data + L-1)

## Test data generation
x_data_test = np.zeros([length_data-L+1,L],dtype = 'float32')
y_data_test = np.zeros([length_data-L+1,1],dtype = 'float32')  ## target data

for i in range(length_data- L + 1):
    y_data_test[i] = y[i + L -1]
    x_data_test[i] = s[i : i + L]   
    
x_data_test = np.concatenate((x_data_test, np.power(x_data_test,3), np.power(x_data_test,5)),axis=1)
test_data = x_data_test

In [ ]:
print(x_data.shape)
print(x_data_test.shape)

Now, with the generated train and test data, estimate $\{h_p[0], h_p[1], h_p[2]\}$ for $p = 1, 3, 5$ using linear regression method.

In [ ]:
import pandas
import seaborn
seaborn.set()

In [ ]:
print(train_data)
print(len(train_data))
print(len(train_data[0])) # 열길이
print(len(train_data[0:])) # 행길이
print(train_data.shape)
type(train_data)

In [ ]:
def linear_regression(params, x_data):
    '''
    The linear regression model in matrix form.
    Arguments:
      params: 1D array of weights for the linear model
      X     : 2D array of input values
    Returns:
      1D array of predicted values
    '''
    return np.dot(x_data, params)

def cost_function(params, model, x_data, y_data):
    '''
    The mean squared error loss function.
    Arguments:
      params: 1D array of weights for the linear model
      model : function for the linear regression model
      X     : 2D array of input values
      y     : 1D array of predicted values
    Returns:
      float, mean squared error
    '''
    y_data_pred = model(params, x_data)
    return np.mean( np.sum((y_data-y_data_pred)**2) )

In [ ]:
from autograd import numpy
from autograd import grad

In [ ]:
gradient = grad(cost_function)

In [ ]:
'''
max_iter = 30
alpha = 0.001
params = numpy.zeros(x_data.shape[1])

for i in range(max_iter):
    descent = gradient(params, linear_regression, x_data, y_data)
    params = params - descent * alpha
    loss = cost_function(params, linear_regression, x_data, y_data)
    if i%5 == 0:
        print("iteration {}, loss = {}".format(i, loss))
'''